<p align="center">
  <img src="techno_volcano.png" />
</p>


In [59]:
import sys
from kubernetes import client, config
import yaml
import requests

def get_yaml_from_git(repo_url,pvc_name):
    response = requests.get(repo_url)
    if response.status_code == 200:
        yaml_content = yaml.safe_load(response.text)
        
        # Assuming the structure of the YAML is consistent with your example,
        # update PVC names for both Master and Worker
        for role in ['Master', 'Worker']:
            if role in yaml_content['spec']['xgbReplicaSpecs']:
                for volume in yaml_content['spec']['xgbReplicaSpecs'][role]['template']['spec']['volumes']:
                    if volume['name'] == 'task-pv-storage':
                        print("updating pvc value to",pvc_name)
                        volume['persistentVolumeClaim']['claimName'] = pvc_name    
        print(yaml_content)
        return yaml_content
    else:
        raise Exception(f"Failed to get YAML file from {repo_url}. Status code: {response.status_code}")
        sys.exit(1)

def get_namespace():
    # Path to the namespace file inside a pod
    namespace_path = '/var/run/secrets/kubernetes.io/serviceaccount/namespace'
    with open(namespace_path, 'r') as file:
        return file.read().strip()

def apply_custom_resource_conditionally(api_instance, group, version, namespace, plural, body):
    name = body['metadata']['name']
    
    try:
        # Retrieve the existing resource, if it exists
        existing_resource = api_instance.get_namespaced_custom_object(
            group=group,
            version=version,
            namespace=namespace,
            plural=plural,
            name=name,
        )
        
        # Check the status of the existing resource
        conditions = existing_resource.get("status", {}).get("conditions", [])
        print(conditions)
        succeeded_condition = next((condition for condition in conditions if condition.get("reason") == "XGBoostJobSucceeded"), None)
        if succeeded_condition and succeeded_condition.get("status") == "True":
            print(f"{name} has Succeeded status, proceeding to delete.")
            # Proceed to delete the resource
            delete_response = api_instance.delete_namespaced_custom_object(
                group=group,
                version=version,
                namespace=namespace,
                plural=plural,
                name=name,
                body=client.V1DeleteOptions(),  # You can provide additional options here
            )
            print(f"Deleted {name}. Delete response: {delete_response}")
        else:
            current_conditions = [f"{condition['type']}: {condition['status']}" for condition in conditions]
            print(f"Existing {name} has not Succeeded. Current conditions: {', '.join(current_conditions) if current_conditions else 'Unknown'}")
            sys.exit(1)  # Exit with error status if the job hasn't succeeded or the condition is not found
    except client.exceptions.ApiException as e:
        if e.status != 404:
            print(f"An unexpected error occurred while checking existing {name}: {e}")
            sys.exit(1)  # Exit with error status if an unexpected API error occurs
        else:
            print(f"{name} does not exist, creating a new one.")
    # If here, either the job doesn't exist or it's safe to replace
    # Proceed with creating or updating the resource
    # This example shows creating/updating without handling the actual logic
    # Replace or update the resource as per your requirement
    try:
        api_response = api_instance.create_namespaced_custom_object(
            group=group,
            version=version,
            namespace=namespace,
            plural=plural,
            body=body,
            field_manager="your-field-manager"  # for server-side apply you need field_manager
        )
        print("Custom resource applied. Status: %s" % api_response)
    except client.exceptions.ApiException as e:
        print(f"Failed to apply {name}: {e}")
        sys.exit(1)

# Example usage
if __name__ == "__main__":
    config.load_incluster_config()
    api_instance = client.CustomObjectsApi()
    yaml_file_url = 'https://raw.githubusercontent.com/chasecadet/kubeflow_course/main/kubeflow_integrations/volcano_iris_train.yml'
    crd_manifest = get_yaml_from_git(yaml_file_url,"xgboostlocal")

    group = 'kubeflow.org'
    version = 'v1'
    plural = 'xgboostjobs'
    # Get namespace
    namespace = get_namespace()
    apply_custom_resource_conditionally(api_instance, group, version, namespace, plural, crd_manifest)



updating pvc value to xgboostlocal
updating pvc value to xgboostlocal
{'apiVersion': 'kubeflow.org/v1', 'kind': 'XGBoostJob', 'metadata': {'name': 'xgboost-dist-iris-test-train-local'}, 'spec': {'xgbReplicaSpecs': {'Master': {'replicas': 1, 'restartPolicy': 'Never', 'template': {'spec': {'volumes': [{'name': 'task-pv-storage', 'persistentVolumeClaim': {'claimName': 'xgboostlocal'}}], 'containers': [{'name': 'xgboost', 'image': 'docker.io/kubeflow/xgboost-dist-iris:latest', 'volumeMounts': [{'name': 'task-pv-storage', 'mountPath': '<PVC_NAME>'}], 'ports': [{'containerPort': 9991, 'name': 'xgboostjob-port'}], 'imagePullPolicy': 'Always', 'args': ['--job_type=Train', '--xgboost_parameter=objective:multi:softprob,num_class:3', '--n_estimators=10', '--learning_rate=0.1', '--model_path=/tmp/xgboost_model/iris_model', '--model_storage_type=local']}]}}}, 'Worker': {'replicas': 3, 'restartPolicy': 'ExitCode', 'template': {'spec': {'volumes': [{'name': 'task-pv-storage', 'persistentVolumeClaim':